# Simplest Iceberg Insert (Spark SQL)

Minimal example that inserts fixed values into an Iceberg table using Spark SQL only.


In [2]:
from pyspark.sql import SparkSession


In [3]:
WAREHOUSE_PATH = "/home/jovyan/work/data/lakehouse/warehouse"
TARGET_TABLE = "local.bronze.fixed_values_sql_v1"

spark = (
    SparkSession.builder
    .appName("iceberg-sql-fixed-insert")
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local.type", "hadoop")
    .config("spark.sql.catalog.local.warehouse", WAREHOUSE_PATH)
    .config("spark.openlineage.columnLineage.enabled", "true")
    .getOrCreate()
)

spark.version


'3.5.3'

In [4]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS local.bronze")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TARGET_TABLE} (
  id INT,
  name STRING,
  status STRING,
  created_at TIMESTAMP
)
USING iceberg
""")

spark.sql(f"""
INSERT INTO {TARGET_TABLE} VALUES
  (1, 'alpha', 'NEW', CURRENT_TIMESTAMP()),
  (2, 'beta', 'NEW', CURRENT_TIMESTAMP()),
  (3, 'gamma', 'DONE', CURRENT_TIMESTAMP())
""")


DataFrame[]

In [5]:
spark.sql(f"SELECT * FROM {TARGET_TABLE} ORDER BY id").show(truncate=False)


+---+-----+------+--------------------------+
|id |name |status|created_at                |
+---+-----+------+--------------------------+
|1  |alpha|NEW   |2026-02-24 12:35:26.336405|
|1  |alpha|NEW   |2026-02-24 12:27:24.914965|
|1  |alpha|NEW   |2026-02-24 13:36:18.135125|
|2  |beta |NEW   |2026-02-24 12:27:24.914965|
|2  |beta |NEW   |2026-02-24 13:36:18.135125|
|2  |beta |NEW   |2026-02-24 12:35:26.336405|
|3  |gamma|DONE  |2026-02-24 13:36:18.135125|
|3  |gamma|DONE  |2026-02-24 12:35:26.336405|
|3  |gamma|DONE  |2026-02-24 12:27:24.914965|
+---+-----+------+--------------------------+



In [6]:
spark.conf.get("spark.openlineage.namespace")

'local-spark'